# **KawsAI - Un Modelo de IA para Matching de ChambeaYa**

En este notebook se guarda el proceso que se hizo para crear el modelo de IA KawsAI. El objetivo de KawsAI, es hacer el matching con la mejor mype a cada estudiante. Así, el estudiante y la mype pueden conectar para poder trabajar juntos.

### **I. Versión:**
1.0

### **II. Objetivo:**
El objetivo de KawsAI es lograr hacer el mejor match entre estudiantes y un puesto ofrecido por una mype. Este match se va a basar en datos que ChambeaYa va a recopilar de ambos lados a traves de su página web. El objetivo principal es lograr indicar a un estudiante cual es el puesto que ofrece una mype que mas coincide con sus conocimientos, y así aumentar mucho las probabilidades de conexión.

### **III. Descripción de los Datasets**
Para la versión 1.0 vamos a usar datasets simulados con la ayuda de la IA de ChatGPT. Se le pidió a ChatGPT simular 2 datasets en forma de tablas y en formato .csv, que son estudiantes_test.csv y mypes_test.csv. Cada una de las columnas de estos datasets contiene información que se va a solicitar a los usuarios ingresar en nuestra plataforma.

##### IIIa. Descripción de columnas de estudiantes:
- **id_estudiante:** ID único del estudiante (string)  
- **nombre_completo:** Nombre completo del estudiante (string)  
- **correo:** Correo electrónico del estudiante (string)  
- **carrera:** Carrera que cursa el estudiante (string)  
- **ciclo_actual:** Ciclo o semestre actual del estudiante (int)  
- **ubicacion:** Ubicación de residencia del estudiante (string)  
- **areas_interes:** Áreas profesionales de interés del estudiante (string)  
- **habilidades_destacadas:** Habilidades más desarrolladas del estudiante (string)  
- **motivacion_principal:** Motivación principal del estudiante (string)  
- **descripcion_personal:** Descripción libre sobre personalidad y hobbies (string)  
- **horas_semanales:** Horas disponibles por semana para trabajar (int)  
- **modalidad_de_trabajo:** Modalidad preferida de trabajo (Presencial, Híbrido, Remoto) (string)  
- **experiencia_relevante:** Experiencia previa relevante (string)  
- **link_portafolio_cv:** Enlace a portafolio o CV del estudiante (string) 

##### IIIb. Descripción de columnas de puestos de mypes:
- **id_puesto:** ID único del puesto ofrecido (string)  
- **titulo_puesto:** Título del puesto ofrecido (string)  
- **descripcion_puesto:** Descripción detallada del puesto (string)  
- **area_del_puesto:** Área o departamento del puesto (string)  
- **modalidad_de_trabajo:** Modalidad del trabajo (Presencial, Híbrido, Remoto) (string)  
- **horas_semanales:** Horas semanales requeridas (int)  
- **fecha_inicio:** Fecha estimada de inicio del puesto (date)  
- **duracion_del_puesto:** Duración del puesto en semanas (int)  
- **sueldo_aproximado:** Sueldo aproximado ofrecido (int)

### **IV. Índice**
1. Carga de Librerias

## 1. Carga de Librerias

In [40]:
# Todas las librerias que se deben importar
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as spacy_stopwords
import requests
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from scipy.sparse import hstack
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

geolocator = Nominatim(user_agent="chambeaya_app")

In [1]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 11.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


## 2. Carga de datos en DataFrames

In [3]:
# Ahora leemos los csv y los guardamos en dataframes de pandas
estudiantes = pd.read_csv("../datasets/estudiantes_test.csv", index_col=False)
puestos = pd.read_csv("../datasets/puestos_test.csv", index_col=False)

## 3. Preprocesamiento de los datos

### 3.1. Para los estudiantes:

#### La función de preprocesamiento va a hacer diferentes pasos para diferentes columnas:

**1)** Vamos a juntar todas las columnas textuales en una sola para el modelo y se le van a dar un peso a cada columna, luego se aplicará tf-idf a todas estas columnas:
- carrera (peso de 1)
- habilidades_destacadas (peso de 3)
- areas_interes (peso de 3)
- motivacion_principal (peso de 1)
- descripcion_personal (peso de 2)
- experiencia_relevante (peso de 5)

**2)** Vamos a aplicar one-hot encoding para la columna modalidad_de_trabajo

**3)** Para las columnas numericas vamos a usar StandardScaler, las columnas son:
- horas_semanales

**4)** Se van a eliminar las siguientes columnas identificadoras que no aportan nada al modelo:
- id_estudiante
- nombre_completo
- correo
- link_portafolio_cv

### 3.2.Para los puestos de trabajo:

#### La clase de preprocesamiento va a tener diferentes preprocesamientos para cada columna

**1)** Para las columnas textuales vamos a juntarlo todo como requerimiento_textual, cada columna textual tendrá un peso
- titulo_puesto
- descripcion_puesto
- area_del_puesto

**2)** Para la columna de modalidad_de_trabajo vamos a aplicar one hot encoding

**3)** Para las columnas numericas como horas_semanales vamos a aplicar Standard Scaler

**4)** Las otras columnas se van a dejar sin usar por ahora ya que no aportan al modelo o son identificadoras

In [36]:
# Se crea una clase preprocesador general
class Preprocesador:
    def __init__(self):
        self.vectorizador = TfidfVectorizer(max_features=500)
        self.scaler= StandardScaler()
        self.columnas_numericas = ['horas_semanales']
        self.columnas_modalidad = []
        self.nlp = spacy.load("es_core_news_sm")

    def crear_perfil_textual(self, text):
        text = re.sub(r"[^\w\s]", " ", text.lower())
        text = re.sub(r"\s+", " ", text).strip()
        doc = self.nlp(text)
        tokens = []
        for token in doc:
            if token.is_stop or token.is_punct or token.like_num:
                continue
            else:
                tokens.append(token.lemma_)
        return " ".join(tokens)

    def crear_perfil_textual_estudiante(self, estudiante):
        return self.crear_perfil_textual(
            ("carrera: " + str(estudiante['carrera']) + " ") * 1 +
            ("habilidades: " + str(estudiante['habilidades_destacadas']) + " ") * 3 +
            ("intereses: " + str(estudiante['areas_interes']) + " ") * 3 +
            ("descripcion: " + str(estudiante['descripcion_personal']) + " ") * 3 +
            ("experiencia: " + str(estudiante['experiencia_relevante']) + " ") * 5
        )

    def crear_perfil_textual_puesto(self, puesto):
        return self.crear_perfil_textual(
            ("puesto: " + str(puesto['titulo_puesto']) + " ") * 1 +
            ("descripcion: " + str(puesto['descripcion_puesto']) + " ") * 5 +
            ("area: " + str(puesto['area_del_puesto']) + " ") * 3
        )

    def fit_transform(self, estudiantes, puestos):
        # Hacemos una copia de los dataframes para no modificarlos directamente
        estudiantes = estudiantes.copy()
        puestos = puestos.copy()

        # Creamos el perfil textual para estudiantes y puestos
        estudiantes['perfil_textual'] = estudiantes.apply(self.crear_perfil_textual_estudiante, axis=1)
        puestos['perfil_textual'] = puestos.apply(self.crear_perfil_textual_puesto, axis=1)

        # Ahora vectorizamos este perfil textual con TF-IDF
        corpus = pd.concat([estudiantes['perfil_textual'], puestos['perfil_textual']], axis=0)
        X_text = self.vectorizador.fit_transform(corpus)

        # Ahora usamos one-hot-encoding para la columna de modalidad
        df_modalidad = pd.concat([estudiantes[['modalidad_de_trabajo']], puestos[['modalidad_de_trabajo']]])
        df_modalidad_encoded = pd.get_dummies(df_modalidad, prefix='modalidad_de_trabajo')
        self.columnas_modalidad = df_modalidad_encoded.columns.tolist()

        # Usamos standard scaler para normalizar las columnas numericas
        columnas_numericas_combinadas = pd.concat([
            estudiantes[self.columnas_numericas],
            puestos[self.columnas_numericas]
        ], axis=0)

        self.scaler.fit(columnas_numericas_combinadas)

        n_estudiantes = len(estudiantes)
        X_puestos = hstack([
            X_text[n_estudiantes:],
            df_modalidad_encoded.iloc[n_estudiantes:].values.astype(np.float32),
            self.scaler.transform(puestos[self.columnas_numericas]).astype(np.float32)
        ])
        X_estudiantes = hstack([
            X_text[:n_estudiantes],
            df_modalidad_encoded.iloc[:n_estudiantes].values.astype(np.float32),
            self.scaler.transform(estudiantes[self.columnas_numericas]).astype(np.float32)
        ])
        return X_estudiantes, X_puestos

    def transform_estudiante(self, estudiante):
        estudiante = estudiante.copy()

        text = self.crear_perfil_textual_estudiante(estudiante.iloc[0])
        X_text = self.vectorizador.transform([text])

        modalidad = pd.get_dummies(pd.DataFrame([estudiante.iloc[0]['modalidad_de_trabajo']], columns=['modalidad_de_trabajo']))
        modalidad = modalidad.reindex(columns=self.columnas_modalidad, fill_value=0)

        num_scaled = self.scaler.transform([estudiante[self.columnas_numericas].iloc[0]])

        return hstack([X_text, modalidad.values.astype(np.float32), num_scaled.astype(np.float32)])


In [ ]:
# Ahora usamos nuestra función de preprocesamiento para realizar el preprocesamiento general, con el fit transform, luego se usara solo transform
pre = Preprocesador()
X_estudiantes, X_puestos = pre.fit_transform(estudiantes, puestos)

## 4. Creación y Entrenamiento del Modelo

Nuestro modelo va a ser KNN para nuestro MVP, vamos a aplicar primero el entrenamiento del KNN con fit, y luego para cada estudiante solo vamos a usar kneighbors y nos devolvera los indices de los puestos ideales para el estudiante

In [39]:
class KawsAIModel:
    def __init__(self):
        self.knn = NearestNeighbors(n_neighbors=5, metric='cosine')
    def train(self, X_puestos):
        self.knn.fit(X_puestos)
    def get_positions(self, X_estudiante_nuevo):
        distances, indexes = self.knn.kneighbors(X_estudiante_nuevo)
        return distances[0], indexes[0]

In [37]:
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(X_puestos)

,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [38]:
estudiante_nuevo = estudiantes.iloc[[16]]
estudiante_nuevo.head()

,id_estudiante,nombre_completo,correo,carrera,ciclo_actual,ubicacion,areas_interes,habilidades_destacadas,motivacion_principal,descripcion_personal,horas_semanales,modalidad_de_trabajo,experiencia_relevante,link_portafolio_cv
16,EST00017,Ana Torres Chávez,ana.torres17@outlook.com,Derecho,6,Huaraz,Derecho Penal,"Constitución, Redacción legal, Oratoria",Estabilidad económica,Buen comunicador,15,Híbrido,Asistente legal en estudio penalista,https://cv.example.com/ana17


In [32]:
X_estudiante_nuevo = pre.transform_estudiante(estudiante_nuevo)

/home/diegomucha/me/programming/projects/chambeaya-ai-core/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [33]:
distancias, indices = knn.kneighbors(X_estudiante_nuevo)
puestos_recomendados = puestos.iloc[indices[0]]

In [34]:
puestos_recomendados

,id_puesto,titulo_puesto,descripcion_puesto,area_del_puesto,modalidad_de_trabajo,horas_semanales,fecha_inicio,duracion_del_puesto,sueldo_aproximado
573,PST00574,Asistente Legal,Apoyo en trámites legales y documentación.,Legal y Derecho,Híbrido,20,2025-06-30,8,1121
941,PST00942,Asistente Legal,Apoyo en trámites legales y documentación.,Legal y Derecho,Híbrido,20,2025-07-09,12,1384
839,PST00840,Asistente Legal,Apoyo en trámites legales y documentación.,Legal y Derecho,Híbrido,20,2025-08-03,12,831
847,PST00848,Asistente Legal,Apoyo en trámites legales y documentación.,Legal y Derecho,Híbrido,20,2025-08-15,4,2886
174,PST00175,Asistente Legal,Apoyo en trámites legales y documentación.,Legal y Derecho,Híbrido,20,2025-07-18,8,2443


In [2]:
estudiantes.head()

NameError: name 'estudiantes' is not defined